In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [10]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

request = LLMRequest(
    conversation=LLMConversation(
        messages=[
            LLMMessage(role='system', content='You are a helpful assistant.'),
            LLMMessage(role='user', content='what is 2+2'),
        ]
    ),
    params=LLMParams(
        model=MODEL_HUB_ID,
        temperature=0,
        response_type=SomeOutput,
        max_completion_tokens=10,
        metadata={'input_id': str(some_input.id)},
    ),
)

batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])

In [6]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

### Apptainer

In [5]:
from pathlib import Path

from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

apptainer_client = infrastructure_container.apptainer_client()
sftp_client = infrastructure_container.sftp_client()
pbs_pro_client = infrastructure_container.pbs_pro_client()
hpc_client = infrastructure_container.hpc_client()

In [6]:
await apptainer_client.health()

2025-04-09 23:08:02,264 - INFO - HTTP Request: GET http://host.docker.internal:7015/health "HTTP/1.1 200 OK"


True

In [7]:
from math_rag.infrastructure.inference.huggingface import HuggingFaceBatchLLM


llm = HuggingFaceBatchLLM(
    remote_project_root=Path('tgi_default_root'),
    hpc_client=hpc_client,
    pbs_pro_client=pbs_pro_client,
    sftp_client=sftp_client,
    apptainer_client=apptainer_client,
)

In [ ]:
await llm.init_resources()

In [8]:
res = await llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_num_retries=0,
)
res

2025-04-09 23:08:04,393 - INFO - Host canonicalization disabled
2025-04-09 23:08:04,394 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22


2025-04-09 23:08:04,419 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-09 23:08:04,419 - INFO - [conn=0]   Local address: 172.22.0.9, port 55338
2025-04-09 23:08:04,420 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-04-09 23:08:04,473 - INFO - [conn=0] Beginning auth for user lpanic
2025-04-09 23:08:05,261 - INFO - [conn=0] Auth for user lpanic succeeded
2025-04-09 23:08:05,264 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-04-09 23:08:05,275 - INFO - [conn=0, chan=0]   Subsystem: sftp
2025-04-09 23:08:05,280 - INFO - [conn=0, chan=0] Starting SFTP client
2025-04-09 23:08:05,468 - INFO - [conn=0, chan=0] Received exit status 0
2025-04-09 23:08:05,470 - INFO - [conn=0, chan=0] Received channel close
2025-04-09 23:08:05,472 - INFO - [conn=0, chan=0] SFTP client exited
2025-04-09 23:08:05,473 - INFO - [conn=0, chan=0] Closing channel
2025-04-09 23:08:05,475 - INFO - [conn=0, chan=0] Channel closed
2025-04-09 23:08:05,476 - INFO 

LLMBatchResult(id=UUID('0d2fe74d-4742-4ff8-af76-4b492a4df6ef'), response_lists=[LLMResponseList(id=UUID('1a4e5d53-06a0-465c-8deb-8009184ed348'), request_id=UUID('edcac871-b0d2-4853-b5fd-9a96740be189'), responses=[LLMResponse(id=UUID('f7f5933a-dd9a-4f5c-b28b-046de3bc81db'), content=BoundAssistantOutput(id=UUID('ac172dc5-e9d6-494c-a789-30cddaf8fb74'), input_id=UUID('ae419d0d-42f4-43e7-ad80-f5da2bdf871b'), result=4), logprobs=None)])], failed_requests=[])

In [11]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'
environ['MAX_RETRIES'] = '3'

%run ../assets/hpc/hf/tgi/tgi.py

2025-04-09 22:36:02,044 - INFO - Reader thread started
2025-04-09 22:36:02,045 - INFO - Processor thread started
2025-04-09 22:36:02,045 - INFO - Writer thread started
2025-04-09 22:36:02,046 - INFO - Finished reading input file
2025-04-09 22:36:02,046 - INFO - Reader thread exiting
2025-04-09 22:36:03,281 - INFO - Processor thread exiting
2025-04-09 22:36:03,282 - INFO - Finished writing output file
2025-04-09 22:36:03,282 - INFO - Writer thread exiting
2025-04-09 22:36:03,283 - INFO - All threads have completed
